In [6]:
import sys
from pathlib import Path

# Caminho absoluto do diretório raiz do projeto
sys.path.append(str(Path().resolve().parent))


In [7]:
import sqlite3
from pathlib import Path
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from indicators.rsi import adicionar_rsi
from indicators.macd import adicionar_macd
from indicators.bollinger import adicionar_bbands
from patterns.engolfo import engolfo_de_alta, engolfo_de_baixa

In [22]:
def carregar_dados():
    # Caminho absoluto até o arquivo sinais.db
    caminho_db = Path().resolve().parent / 'data' / 'sinais.db'
    
    # Conecta e carrega os dados
    conn = sqlite3.connect(str(caminho_db))
    df = pd.read_sql('SELECT * FROM candles ORDER BY open_time', conn, parse_dates=['open_time'])
    conn.close()
    return df
    
def preparar_dados(df):
    print("Antes do preparo:", len(df))
    df = adicionar_rsi(adicionar_macd(adicionar_bbands(df)))
    df['engolfo_alta'] = False
    df['engolfo_baixa'] = False
    for i in range(1, len(df)):
        df.loc[df.index[i], 'engolfo_alta'] = engolfo_de_alta(df.iloc[i], df.iloc[i-1])
        df.loc[df.index[i], 'engolfo_baixa'] = engolfo_de_baixa(df.iloc[i], df.iloc[i-1])
    df['target'] = (df['close'].shift(-1) > df['close']).astype(int)
    df = df.dropna()
    print("Depois do preparo:", len(df))
    return df


def treinar_modelo(df):
    features = ['open','high','low','close','volume','rsi','macd','macd_signal','macd_diff','bb_high','bb_low','engolfo_alta','engolfo_baixa']
    X = df[features]
    y = df['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    modelo = RandomForestClassifier(n_estimators=100, random_state=42)
    modelo.fit(X_train, y_train)
    pred = modelo.predict(X_test)
    print('Accuracy:', accuracy_score(y_test, pred))
    print('Precision:', precision_score(y_test, pred))
    print('Recall:', recall_score(y_test, pred))
    print('F1:', f1_score(y_test, pred))
    print('Matriz de confusao:\n', confusion_matrix(y_test, pred))
    Path('models').mkdir(exist_ok=True)
    joblib.dump(modelo, 'models/modelo_binario.pkl')
    joblib.dump(scaler, 'models/escalador.pkl')
    return modelo, scaler

df = carregar_dados()
df = preparar_dados(df)
treinar_modelo(df)

Antes do preparo: 26
Depois do preparo: 0


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [19]:
df_raw = carregar_dados()
print(len(df_raw))
df_raw.head()


14


,id,ativo,open_time,open,high,low,close,volume
0,1,BTCUSDT,2025-06-18 22:51:00,104910.56,104915.98,104910.55,104915.98,0.53467
1,2,BTCUSDT,2025-06-18 22:56:00,104881.87,104881.87,104868.00,104868.01,1.38572
2,3,BTCUSDT,2025-06-18 22:59:00,104795.20,104795.21,104795.20,104795.20,0.09474
3,4,BTCUSDT,2025-06-18 22:59:00,104795.20,104803.34,104795.20,104803.34,0.99211
4,5,BTCUSDT,2025-06-18 23:00:00,104815.23,104830.40,104815.23,104830.39,0.64813
